# This notebook is making txt files of the mass in the disk and the CGM

In [ ]:
######################################################
# Script for getting the mass of the CGM and disk
# Designed to run on one node so don't lose info
######################################################

# imports
import yt
import numpy as np

# creating the time series
ts = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf/DD????/DD????")

# create the master list for each mass
disk_total_mass_list = []
disk_gas_mass_list = []
disk_part_mass_list = []

CGM_total_mass_list = []
CGM_gas_mass_list = []
CGM_part_mass_list = []

CGM2_total_mass_list = []
CGM2_gas_mass_list = []
CGM2_part_mass_list = []

# create master time list
time_list = []

# main loop
for ds in ts.piter():
    
    # create the cylinder for the disk
    disk_cyl = ds.disk([0.5,0.5,0.5], [0,0,1], (20,'kpc'), (5,'kpc'))
    # create the sphere for the CGM within the virial radius
    CGM_sph = ds.sphere([0.5,0.5,0.5], (206, 'kpc'))
    CGM2_sph = ds.sphere([0.5,0.5,0.5], (412, 'kpc'))
    
    # making sure there are particles to run it
    # if not, set particle mass to zero
    if ds.particles_exist == False:
        # calculate masses for the disk first
        disk_baryon_mass = disk_cyl.quantities.total_quantity(("gas", "mass"))
        disk_particle_mass = 0
    
        disk_total_mass = disk_baryon_mass.in_units('Msun')
        disk_gas_mass = disk_baryon_mass.in_units('Msun')
        
        # now can calulcate CGM quantities by subtracting the masses of the disk
        CGM_baryon_mass = CGM_sph.quantities.total_quantity(("gas", "mass"))
        CGM_particle_mass = 0
    
        CGM_total_mass = CGM_baryon_mass.in_units('Msun') - disk_total_mass
        CGM_gas_mass = CGM_baryon_mass.in_units('Msun') - disk_gas_mass
    
        CGM2_baryon_mass = CGM2_sph.quantities.total_quantity(("gas", "mass"))
        CGM2_particle_mass = 0
    
        CGM2_total_mass = CGM2_baryon_mass.in_units('Msun') - disk_total_mass
        CGM2_gas_mass = CGM2_baryon_mass.in_units('Msun') - disk_gas_mass
        
        # append quantities to the master lists
        disk_total_mass_list.append(disk_total_mass)
        disk_gas_mass_list.append(disk_gas_mass)
        disk_part_mass_list.append(disk_particle_mass)
    
        CGM_total_mass_list.append(CGM_total_mass)
        CGM_gas_mass_list.append(CGM_gas_mass)
        CGM_part_mass_list.append(CGM_particle_mass)
    
        CGM2_total_mass_list.append(CGM2_total_mass)
        CGM2_gas_mass_list.append(CGM2_gas_mass)
        CGM2_part_mass_list.append(CGM2_particle_mass)
    
        current_time = ds.current_time.to('Gyr')
        time_list.append(current_time)
    else:
        # calculate masses for the disk first
        disk_baryon_mass, disk_particle_mass = disk_cyl.quantities.total_quantity(
        [("gas", "mass"), ("all", "particle_mass")])
    
        disk_total_mass = (disk_baryon_mass + disk_particle_mass).in_units('Msun')
        disk_gas_mass = disk_baryon_mass.in_units('Msun')
        disk_part_mass = disk_particle_mass.in_units('Msun')
    
        # now can calulcate CGM quantities by subtracting the masses of the disk
        CGM_baryon_mass, CGM_particle_mass = CGM_sph.quantities.total_quantity(
        [("gas", "mass"), ("all", "particle_mass")])
    
        CGM_total_mass = (CGM_baryon_mass + CGM_particle_mass).in_units('Msun') - disk_total_mass
        CGM_gas_mass = CGM_baryon_mass.in_units('Msun') - disk_gas_mass
        CGM_part_mass = CGM_particle_mass.in_units('Msun') - disk_part_mass
    
        CGM2_baryon_mass, CGM2_particle_mass = CGM2_sph.quantities.total_quantity(
        [("gas", "mass"), ("all", "particle_mass")])
    
        CGM2_total_mass = (CGM2_baryon_mass + CGM2_particle_mass).in_units('Msun') - disk_total_mass
        CGM2_gas_mass = CGM2_baryon_mass.in_units('Msun') - disk_gas_mass
        CGM2_part_mass = CGM2_particle_mass.in_units('Msun') - disk_part_mass
    
        # append quantities to the master lists
        disk_total_mass_list.append(disk_total_mass)
        disk_gas_mass_list.append(disk_gas_mass)
        disk_part_mass_list.append(disk_part_mass)
    
        CGM_total_mass_list.append(CGM_total_mass)
        CGM_gas_mass_list.append(CGM_gas_mass)
        CGM_part_mass_list.append(CGM_part_mass)
    
        CGM2_total_mass_list.append(CGM2_total_mass)
        CGM2_gas_mass_list.append(CGM2_gas_mass)
        CGM2_part_mass_list.append(CGM2_part_mass)
    
        current_time = ds.current_time.to('Gyr')
        time_list.append(current_time)
    
    
# now that have data can create txt files
# first turn to numpy arrays, then stack, then save
disk_total_mass_array = np.array(disk_total_mass_list)
disk_gas_mass_array = np.array(disk_gas_mass_list)
disk_part_mass_array = np.array(disk_part_mass_list)

CGM_total_mass_array = np.array(CGM_total_mass_list)
CGM_gas_mass_array = np.array(CGM_gas_mass_list)
CGM_part_mass_array = np.array(CGM_part_mass_list)

CGM2_total_mass_array = np.array(CGM2_total_mass_list)
CGM2_gas_mass_array = np.array(CGM2_gas_mass_list)
CGM2_part_mass_array = np.array(CGM2_part_mass_list)

time_array = np.array(time_list)

disk_mass_data = np.column_stack((time_array,disk_total_mass_array,disk_gas_mass_array,disk_part_mass_array))
CGM_mass_data = np.column_stack((time_array,CGM_total_mass_array,CGM_gas_mass_array,CGM_part_mass_array,
                                 CGM2_total_mass_array,CGM2_gas_mass_array,CGM2_part_mass_array))

# Save the combined data to a text file
np.savetxt('newCGM_disk_mass_redo.txt', disk_mass_data, delimiter=',', header='time,total mass,gas mass,particle mass', 
           comments='', fmt='%.5e')

np.savetxt('newCGM_CGM_mass_redo.txt', CGM_mass_data, delimiter=',', 
           header='time,CGM total mass,CGM gas mass,CGM particle mass,CGM2 total mass,CGM2 gas mass,CGM2 particle mass', 
           comments='', fmt='%.5e')